I have a list of six latin american countries that I would like to pair to a set of countries in terms of the goods they export. I am doing that at the ISIC rev. 4 level. To do that, I have downloaded UN Comtrade data for the year 1995.

This is what I do in this notebook:

1) I import all the data into four different datasets (imports and exports, and latam countries and rest of the world).
2) I convert all the trade data (on HS classification) to ISIC classification
3) I compute the share of exports and imports on each sector within each country.
4) I use cosine similarity, euclidean distances and pearson correlations to get a list of the 5 countries that show more similarity for each latin american country in terms of imports and exports.

In [1]:
import pandas as pd
import os

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr

In [3]:
# Initializing dataframes
exports_world=pd.DataFrame()
imports_world=pd.DataFrame()
imports_latam=pd.DataFrame()
exports_latam=pd.DataFrame()
#directory with the data
directory='Trade_Data/'
#filenames within the directory
filenames = os.listdir(directory)
# opening files + keeping relevant columns + splitting exports and imports + appending them 
for filename in filenames:
    df=pd.read_csv(directory+filename , encoding='latin-1')[['ReporterDesc', 'FlowCode', 'Partner2Desc', 'ClassificationCode', 'CmdCode', 'PrimaryValue']]
    if filename.startswith( 'World' ):
        exports_world=pd.concat([exports_world, df[df['FlowCode']=='X']], axis=0).reset_index(drop=True)
        imports_world=pd.concat([imports_world,df[df['FlowCode']=='M']], axis=0).reset_index(drop=True)
    else:
        exports_latam=pd.concat([exports_latam,df[df['FlowCode']=='X']], axis=0).reset_index(drop=True)
        imports_latam=pd.concat([imports_latam,df[df['FlowCode']=='M']], axis=0).reset_index(drop=True)

In [4]:
# Uruguay is in the dataset when it should not be there
exports_world=exports_world[exports_world['ReporterDesc']!='Uruguay']
imports_world=imports_world[imports_world['ReporterDesc']!='Uruguay']

In [5]:
# To ISIC
conversion=pd.read_excel('ConversionKeyBTDIxE4PUB.xlsx', sheet_name='FromHSToISICToEC')
# Keep HS 0
conversion=conversion[conversion['HS']==0]

C:\Users\guill\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [6]:
# Remove last character from 'Desci4' if its length is 4, and remove ' used' if the last 5 characters are ' used'
conversion['Desci4'] = conversion['Desci4'].apply(lambda x: x[:-1] if len(x) == 4 else x)
conversion['Desci4'] = conversion['Desci4'].apply(lambda x: x[:-5] if x[-5:] == ' used' else x)

# Replace 'Desci4' values with their corresponding replacements
conversion['isic4'] = conversion['Desci4'].replace({'D01': 'D01T02', 'D02': 'D01T02', 'D05': 'D05T06', 
                                             'D06': 'D05T06','D07': 'D07T08', 'D08': 'D07T08', 'D13':'D13T15', 
                                             'D14':'D13T15', 'D15':'D13T15', 'D17':'D17T18', 'D18':'D17T18', 
                                             'D31':'D31T33', 'D32':'D31T33', 'D38':'D36T39','D58':'D58T60', 'D59':'D58T60', 
                                            'D10': 'D10T12', 'D11': 'D10T12', 'D12': 'D10T12', 'D241T31': 'D24','D242T32': 'D24'})

# Convert string variable to numeric
conversion['HS-6digit']=pd.to_numeric(conversion['HS-6digit'], errors='coerce')

In [7]:
def merge_with_conversion(dataset):
    return pd.merge(conversion[['HS-6digit', 'isic4']].dropna(subset=['HS-6digit']), dataset, left_on='HS-6digit', right_on='CmdCode', how='right')

def create_shares(dataset):
    data1= dataset[['ReporterDesc','isic4', 'PrimaryValue']].groupby(['ReporterDesc','isic4']).sum().reset_index()
    data2=dataset[['ReporterDesc', 'PrimaryValue']].groupby(['ReporterDesc']).sum().reset_index().rename(columns={"PrimaryValue": "total"})
    data1=pd.merge(data2, data1, on='ReporterDesc')
    data1['shares']=data1['PrimaryValue']/data1['total']
    return data1

def pivot_shares(dataset):
    return dataset.pivot(index='ReporterDesc', columns='isic4', values='shares').fillna(0)

In [8]:
exports_pivoted_world = pivot_shares(create_shares(merge_with_conversion(exports_world)))
imports_pivoted_world = pivot_shares(create_shares(merge_with_conversion(imports_world)))
exports_pivoted_latam = pivot_shares(create_shares(merge_with_conversion(exports_latam)))
imports_pivoted_latam = pivot_shares(create_shares(merge_with_conversion(imports_latam)))

In [9]:
# Function to compute cosine similarity
def compute_similarity(df1, df2):
    # Align the columns of both dataframes
    df1_aligned, df2_aligned = df1.align(df2, axis=1, fill_value=0)
    # Compute cosine similarity
    similarity_matrix = cosine_similarity(df1_aligned, df2_aligned)
    return pd.DataFrame(similarity_matrix, index=df1.index, columns=df2.index)

# Function to compute Euclidean distance
def compute_euclidean(df1, df2):
    # Align the columns of both dataframes
    df1_aligned, df2_aligned = df1.align(df2, axis=1, fill_value=0)

    # Initialize a dataframe to store Euclidean distances
    euclidean_distances = pd.DataFrame(index=df1.index, columns=df2.index)

    # Compute Euclidean distance for each pair of countries
    for latam_country in df1_aligned.index:
        for world_country in df2_aligned.index:
            distance = euclidean(df1_aligned.loc[latam_country], df2_aligned.loc[world_country])
            euclidean_distances.at[latam_country, world_country] = distance

    return euclidean_distances

# Function to compute Pearson correlation
def compute_pearson(df1, df2):
    # Align the columns of both dataframes
    df1_aligned, df2_aligned = df1.align(df2, axis=1, fill_value=0)

    # Initialize a dataframe to store Pearson correlation coefficients
    pearson_correlations = pd.DataFrame(index=df1.index, columns=df2.index)

    # Compute Pearson correlation for each pair of countries
    for latam_country in df1_aligned.index:
        for world_country in df2_aligned.index:
            corr, _ = pearsonr(df1_aligned.loc[latam_country], df2_aligned.loc[world_country])
            pearson_correlations.at[latam_country, world_country] = corr

    return pearson_correlations

In [10]:
# Compute cosine similarity
similarity = compute_similarity(exports_pivoted_latam, exports_pivoted_world)

# Display similarity for each LATAM country in an ordered table
ordered_similarity_tables = {}
for country in similarity.index:
    ordered_similarity_tables[country] = similarity.loc[country].sort_values(ascending=False)

# Compute Euclidean distance
euclidean_distances = compute_euclidean(exports_pivoted_latam, exports_pivoted_world)

# Compute Pearson correlation
pearson_correlations = compute_pearson(exports_pivoted_latam, exports_pivoted_world)

# Display Euclidean distances and Pearson correlations for each LATAM country in an ordered table
ordered_euclidean_tables = {country: euclidean_distances.loc[country].sort_values() for country in euclidean_distances.index}
ordered_pearson_tables = {country: pearson_correlations.loc[country].sort_values(ascending=False) for country in pearson_correlations.index}

In [11]:
cosine_exports=pd.DataFrame([[i for i in ordered_similarity_tables[j][0:5].index] for j in ordered_similarity_tables], index=[j for j in ordered_similarity_tables])
euclidean_exports=pd.DataFrame([[i for i in ordered_euclidean_tables[j][0:5].index] for j in ordered_euclidean_tables], index=[j for j in ordered_euclidean_tables])
pearson_exports=pd.DataFrame([[i for i in ordered_pearson_tables[j][0:5].index] for j in ordered_pearson_tables], index=[j for j in ordered_pearson_tables])


In [12]:
cosine_exports

,0,1,2,3,4
Argentina,Malawi,New Zealand,Gambia,Cyprus,Rep. of Moldova
Brazil,Zimbabwe,New Zealand,Hungary,Lithuania,Estonia
Chile,Australia,Zambia,Zimbabwe,North Macedonia,Slovakia
Colombia,Cameroon,Egypt,Indonesia,Ethiopia,Côte d'Ivoire
Costa Rica,Uganda,Côte d'Ivoire,Burkina Faso,Ethiopia,Madagascar
Mexico,Japan,United Kingdom,Spain,Canada,USA


In [13]:
euclidean_exports

,0,1,2,3,4
Argentina,New Zealand,Cyprus,Gambia,Zimbabwe,Greece
Brazil,Hungary,Estonia,Lithuania,Zimbabwe,Australia
Chile,Australia,Zimbabwe,North Macedonia,Slovakia,Kyrgyzstan
Colombia,Cameroon,Egypt,Indonesia,Australia,Kyrgyzstan
Costa Rica,Côte d'Ivoire,Madagascar,Kiribati,Uganda,Burkina Faso
Mexico,United Kingdom,Canada,USA,Spain,France


In [14]:
pearson_exports

,0,1,2,3,4
Argentina,Malawi,New Zealand,Gambia,Cyprus,Rep. of Moldova
Brazil,Zimbabwe,New Zealand,Iceland,Rep. of Moldova,Hungary
Chile,Zambia,Australia,Zimbabwe,North Macedonia,Slovakia
Colombia,Cameroon,Egypt,Ethiopia,Uganda,Burkina Faso
Costa Rica,Uganda,Côte d'Ivoire,Burkina Faso,Ethiopia,Madagascar
Mexico,Japan,United Kingdom,Spain,Canada,USA


In [15]:
# Compute cosine similarity
similarity = compute_similarity(imports_pivoted_latam, imports_pivoted_world)

# Display similarity for each LATAM country in an ordered table
ordered_similarity_tables = {}
for country in similarity.index:
    ordered_similarity_tables[country] = similarity.loc[country].sort_values(ascending=False)

# Compute Euclidean distance
euclidean_distances = compute_euclidean(imports_pivoted_latam, imports_pivoted_world)

# Compute Pearson correlation
pearson_correlations = compute_pearson(imports_pivoted_latam, imports_pivoted_world)

# Display Euclidean distances and Pearson correlations for each LATAM country in an ordered table
ordered_euclidean_tables = {country: euclidean_distances.loc[country].sort_values() for country in euclidean_distances.index}
ordered_pearson_tables = {country: pearson_correlations.loc[country].sort_values(ascending=False) for country in pearson_correlations.index}

In [16]:
cosine_imports=pd.DataFrame([[i for i in ordered_similarity_tables[j][0:5].index] for j in ordered_similarity_tables], index=[j for j in ordered_similarity_tables])
euclidean_imports=pd.DataFrame([[i for i in ordered_euclidean_tables[j][0:5].index] for j in ordered_euclidean_tables], index=[j for j in ordered_euclidean_tables])
pearson_imports=pd.DataFrame([[i for i in ordered_pearson_tables[j][0:5].index] for j in ordered_pearson_tables], index=[j for j in ordered_pearson_tables])


In [17]:
cosine_imports

,0,1,2,3,4
Argentina,New Zealand,Zimbabwe,Australia,Sweden,United Rep. of Tanzania
Brazil,New Zealand,France,Spain,Netherlands,Sweden
Chile,Zimbabwe,Czechia,New Zealand,Poland,Zambia
Colombia,Indonesia,Zimbabwe,Norway,Italy,Sweden
Costa Rica,United Rep. of Tanzania,Poland,Italy,Indonesia,France
Mexico,Sweden,Finland,Australia,Denmark,Ireland


In [18]:
euclidean_imports

,0,1,2,3,4
Argentina,New Zealand,Zimbabwe,Australia,Sweden,United Rep. of Tanzania
Brazil,New Zealand,France,Spain,Netherlands,Sweden
Chile,Zimbabwe,Czechia,New Zealand,Poland,Zambia
Colombia,Zimbabwe,Indonesia,Norway,Italy,Sweden
Costa Rica,United Rep. of Tanzania,Poland,Italy,France,Hungary
Mexico,Sweden,Finland,Denmark,Australia,United Kingdom


In [19]:
pearson_imports

,0,1,2,3,4
Argentina,New Zealand,Zimbabwe,Australia,Sweden,United Rep. of Tanzania
Brazil,New Zealand,France,Spain,Netherlands,Sweden
Chile,Zimbabwe,Czechia,Poland,New Zealand,Zambia
Colombia,Indonesia,Zimbabwe,Norway,Italy,Thailand
Costa Rica,United Rep. of Tanzania,Indonesia,Italy,Poland,France
Mexico,Sweden,Ireland,Finland,Australia,Denmark
